In [3]:
import pandas as pd
import numpy as np

def calculate_skew_kurt(window):
    """Manual calculation of skewness and kurtosis using NumPy (CPU-based)"""
    # Ensure float32 for efficiency
    window = window.astype(np.float32)
    
    # Calculate moments
    mean = np.mean(window, axis=0)
    std = np.std(window, axis=0, ddof=0)
    diffs = window - mean
    
    # Avoid division by zero
    std += np.where(std == 0, 1e-8, 0)
    
    # Standardized values
    z = diffs / std
    
    # Calculate skewness and kurtosis
    skew = np.mean(z**3, axis=0)
    kurt = np.mean(z**4, axis=0) - 3  # Fisher's definition
    
    return skew, kurt

def extract_time_features(window, eeg_columns):
    """CPU-accelerated feature extraction with manual skew/kurtosis"""
    features = {}
    
    # Basic stats
    features.update({
        f'{col}_mean': val for col, val in zip(
            eeg_columns, np.mean(window, axis=0).tolist())
    })
    features.update({
        f'{col}_std': val for col, val in zip(
            eeg_columns, np.std(window, axis=0).tolist())
    })
    features.update({
        f'{col}_var': val for col, val in zip(
            eeg_columns, np.var(window, axis=0).tolist())
    })
    
    # Signal characteristics
    features.update({
        f'{col}_max': val for col, val in zip(
            eeg_columns, np.max(window, axis=0).tolist())
    })
    features.update({
        f'{col}_min': val for col, val in zip(
            eeg_columns, np.min(window, axis=0).tolist())
    })
    features.update({
        f'{col}_ptp': val for col, val in zip(
            eeg_columns, np.ptp(window, axis=0).tolist())
    })
    
    # Manual skew/kurt calculation
    skew_vals, kurt_vals = calculate_skew_kurt(window)
    features.update({
        f'{col}_skew': val for col, val in zip(eeg_columns, skew_vals.tolist())
    })
    features.update({
        f'{col}_kurt': val for col, val in zip(eeg_columns, kurt_vals.tolist())
    })
    
    # Zero-crossing
    sign_changes = np.diff((window >= 0).astype(int), axis=0)
    features.update({
        f'{col}_zcross': val for col, val in zip(
            eeg_columns, np.sum(sign_changes != 0, axis=0).tolist())
    })
    
    # Energy features
    features.update({
        f'{col}_energy': val for col, val in zip(
            eeg_columns, np.sum(window**2, axis=0).tolist())
    })
    features.update({
        f'{col}_rms': val for col, val in zip(
            eeg_columns, np.sqrt(np.mean(window**2, axis=0)).tolist())
    })
    
    return features

def sliding_window_features(eeg_data, outcomes, eeg_columns, window_size, step_size):
    """CPU-based sliding window feature extraction with target alignment."""
    features_list = []
    targets = []
    n_samples = eeg_data.shape[0]
    
    for start in range(0, n_samples - window_size + 1, step_size):
        end = start + window_size
        window = eeg_data[start:end]
        outcome_window = outcomes[start:end]
        
        features = extract_time_features(window, eeg_columns)
        features_list.append(features)
        
        # Determine window target (modify based on your use case)
        targets.append(1 if np.any(outcome_window) else 0)
    
    return features_list, targets

# Main processing
if __name__ == "__main__":
    # Load data with proper dtype specification
    data = pd.read_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/EEG Assets/chbmit_preprocessed_data.csv')
    eeg_columns = [col for col in data.columns if col != 'Outcome']
    
    # Convert to NumPy arrays with float32 precision
    eeg_data = np.asarray(data[eeg_columns].values, dtype=np.float32)
    outcomes = np.asarray(data['Outcome'].values, dtype=np.float32)
    
    # Window parameters
    fs = 256  # Sampling frequency
    window_size = fs * 1  # 1-second windows
    step_size = window_size // 2  # 50% overlap
    
    # Process data on CPU
    features, targets = sliding_window_features(
        eeg_data, outcomes, eeg_columns, window_size, step_size
    )
    
    # Convert to DataFrame and save
    features_df = pd.DataFrame(features)
    features_df['target'] = targets
    features_df.to_csv('/Users/puchku-home/Downloads/TIME FEATURES/timefeature_data.csv', index=False)
    print("Feature extraction complete. Features saved to 'timefeature_data.csv'")


Feature extraction complete. Features saved to 'timefeature_data.csv'


### Added Hjorth Parameters,Fractal & Nonlinear Features(HURST). Also modified the mean and other values little bit not too big of change in existing features. Will be considering the 1st code output for model training.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
from scipy.signal import welch
from antropy import app_entropy, sample_entropy
from hurst import compute_Hc

# ---------------- Feature Calculation Functions ---------------- #

def calculate_hjorth(window):
    """Compute Hjorth mobility and complexity safely."""
    first_derivative = np.diff(window, axis=0)
    second_derivative = np.diff(first_derivative, axis=0)

    var_zero = np.var(window, axis=0)
    var_d1 = np.var(first_derivative, axis=0)
    var_d2 = np.var(second_derivative, axis=0)

    # Prevent division by zero
    var_zero = np.where(var_zero == 0, 1e-8, var_zero)
    var_d1 = np.where(var_d1 == 0, 1e-8, var_d1)

    mobility = np.sqrt(var_d1 / var_zero)
    complexity = np.sqrt(var_d2 / var_d1) / mobility

    return mobility, complexity

def calculate_hurst(window):
    """Calculate Hurst Exponent safely."""
    def safe_hurst(x):
        try:
            H, _, _ = compute_Hc(x, kind='change', simplified=True)
            return H
        except:
            return np.nan  # Return NaN if calculation fails

    return np.apply_along_axis(safe_hurst, axis=0, arr=window)

def extract_time_features(window, eeg_columns):
    """Extract time-domain features."""
    features = {}

    # Basic stats
    features.update({f'{col}_mean': val for col, val in zip(eeg_columns, np.mean(window, axis=0))})
    features.update({f'{col}_std': val for col, val in zip(eeg_columns, np.std(window, axis=0))})
    features.update({f'{col}_rms': val for col, val in zip(eeg_columns, np.sqrt(np.mean(window**2, axis=0)))})

    # Hjorth parameters
    mobility_vals, complexity_vals = calculate_hjorth(window)
    features.update({f'{col}_mobility': val for col, val in zip(eeg_columns, mobility_vals)})
    features.update({f'{col}_complexity': val for col, val in zip(eeg_columns, complexity_vals)})

    # Hurst exponent
    features.update({f'{col}_hurst': val for col, val in zip(eeg_columns, calculate_hurst(window))})

    return features

def sliding_window_features(eeg_data, outcomes, eeg_columns, window_size, step_size):
    """Extract features using a sliding window."""
    features_list = []
    targets_list = []

    for start in range(0, len(eeg_data) - window_size + 1, step_size):
        end = start + window_size
        window = eeg_data[start:end]
        outcome_window = outcomes[start:end]

        # Extract features
        features = extract_time_features(window, eeg_columns)
        features_list.append(features)

        # Assign target: 1 if any seizure in the window, else 0
        targets_list.append(int(np.any(outcome_window == 1)))

    return features_list, targets_list

# ---------------- Main Processing ---------------- #

# Load dataset (assuming it's preprocessed and available as a CSV)
df = pd.read_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/EEG Assets/chbmit_preprocessed_data.csv')

# Separate EEG data and outcomes
eeg_columns = [col for col in df.columns if col != "Outcome"]
eeg_data = df[eeg_columns].values
outcomes = df["Outcome"].values

# Define window and step size
window_size = 256  # Adjust as needed
step_size = window_size // 2  # 50% overlap

# Extract features
features, targets = sliding_window_features(eeg_data, outcomes, eeg_columns, window_size, step_size)

# Convert to DataFrame and save
features_df = pd.DataFrame(features)
features_df["Outcome"] = targets
features_df.to_csv('/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME FEATURES/timefeature_data1.csv', index=False)

print("Feature extraction completed. Saved as '/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME FEATURES/timefeature_data1.csv'.")


/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Feature extraction completed. Saved as '/Users/puchku-home/KIIT SEM/PROJECT/EEG/TIME FEATURES/timefeature_data1.csv'.
